In [39]:
import sys

# This package is not configured properly, I think.
# Without this line, all methods & submodules will fail to import
sys.path.append('./abstract_algebra/src')

from typing import Union
from functools import reduce, partial
from random import randint
import pathlib
import polars as pl
import os


from abstract_algebra.src.finite_algebras import (
  generate_symmetric_group,
  generate_cyclic_group,
)

In [3]:
data_dir = pathlib.Path('./data')
if not os.path.exists(data_dir):
  os.mkdir(data_dir)

In [4]:
symmetric_groups = [
  generate_symmetric_group(1),
  generate_symmetric_group(2),
  generate_symmetric_group(3),
  generate_symmetric_group(4),
  generate_symmetric_group(5),
  generate_symmetric_group(6),
]

cyclic_groups = [
  generate_cyclic_group(1),
  generate_cyclic_group(2),
  generate_cyclic_group(3),
  generate_cyclic_group(4),
  generate_cyclic_group(5),
  generate_cyclic_group(6),
]

product_groups = [
  cyclic_groups[4] * cyclic_groups[5],
  cyclic_groups[5] * cyclic_groups[5],
  symmetric_groups[1] * cyclic_groups[3],
  symmetric_groups[2] * cyclic_groups[3],
  symmetric_groups[3] * cyclic_groups[3],
  symmetric_groups[4] * cyclic_groups[3],
]



S5 = generate_symmetric_group(
  5, 
  name="S5", 
  description="Symmetric group on 5 elements")
S5.about(use_table_names=True)


** Group **
Name: S5
Instance ID: 4587105936
Description: Symmetric group on 5 elements
Order: 120
Identity: (1, 2, 3, 4, 5)
Commutative? No
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0 (1, 2, 3, 4, 5) (1, 2, 3, 4, 5)       1
      1 (1, 2, 3, 5, 4) (1, 2, 3, 5, 4)       2
      2 (1, 2, 4, 3, 5) (1, 2, 4, 3, 5)       2
      3 (1, 2, 4, 5, 3) (1, 2, 5, 3, 4)       3
      4 (1, 2, 5, 3, 4) (1, 2, 4, 5, 3)       3
      5 (1, 2, 5, 4, 3) (1, 2, 5, 4, 3)       2
      6 (1, 3, 2, 4, 5) (1, 3, 2, 4, 5)       2
      7 (1, 3, 2, 5, 4) (1, 3, 2, 5, 4)       2
      8 (1, 3, 4, 2, 5) (1, 4, 2, 3, 5)       3
      9 (1, 3, 4, 5, 2) (1, 5, 2, 3, 4)       4
     10 (1, 3, 5, 2, 4) (1, 4, 2, 5, 3)       4
     11 (1, 3, 5, 4, 2) (1, 5, 2, 4, 3)       3
     12 (1, 4, 2, 3, 5) (1, 3, 4, 2, 5)       3
     13 (1, 4, 2, 5, 3) (1, 3, 5, 2, 4)       4
     14 (1, 4, 3, 2, 5) (1, 4, 3, 2, 5)       2
     15 (1, 4, 3, 5, 2) (1, 5, 3, 2, 4)       3
     16 (1, 4, 5, 2, 3) (1, 4, 5, 2

In [38]:
def group_reduce(lhs: Union[str, int], rhs: int, G) -> int:
  if isinstance(lhs, str):
    prod = G.op(lhs, G.elements[rhs])
  else:
    prod = G.op(G.elements[lhs], G.elements[rhs])
  
  return G.elements.index(prod)


examples = []
num_examples = 10_000
max_len = 20
for prod_len in range(1, max_len+1):
  for i in range(num_examples):
    lhs = []
    for _ in range(prod_len):
      lhs.append(randint(0, len(S5.elements)-1))
    examples.append({"length": len(lhs), "input": " ".join(map(str, lhs)), "target": str(reduce(partial(group_reduce, G=S5), lhs))})

In [6]:
ex_df = pl.from_dicts(examples)

for i in range(1, max_len+1):
  ex_df.filter(pl.col("length") == i).write_csv(data_dir / f"S5_{i}.csv")

In [7]:
ex_df.tail()

length,input,target
i64,str,str
20,"""26 11 4 115 75…","""21"""
20,"""101 104 15 23 …","""53"""
20,"""92 22 58 114 1…","""15"""
20,"""89 103 91 5 11…","""83"""
20,"""55 12 23 98 90…","""1"""


In [33]:
test = pl.read_csv(data_dir / "S5_7.csv")
test.with_columns(
  pl.concat_str([pl.col("input"), pl.col("target")], separator=" ").alias("merged")
).select(pl.col("merged").map_batches(
  lambda x: x.str.split(" ")
)).explode("merged").unique().shape[0]

120